# <u>Estimating Logical Error Rates for the Surface Code</u>

## **Part 1: Setting Up the Stabilizer Code** 

The $L=3$ (unrotated) surface code arranges qubits on a grid as follows:

<img src="./Figs/LER-surfacequbits.svg" width="300">

Note the labeling of the qubits in purple.

* We have $n=13$ physical qubits here.
* Additionally, it turns out that this code will encode $k=1$ logical qubit and has a code distance of $d=3$ (we don't worry about this analysis).
* We call this a $\llbracket 13, 1, 3 \rrbracket$ quantum (CSS) code.

We create such a code with a `StabCode` object:

In [1]:
from scalerqec.QEC.qeccircuit import StabCode

surface_code = StabCode(n=13, k=1, d=3)

The stabilizers of the code are defined as follows:

<img src="./Figs/LER-surfacestabs.svg" width="300">

We can then read off the expressions for the $X$ and $Z$ stabilizers:
* $X$ stabilizers
    * $X \otimes I \otimes I \otimes X \otimes I \otimes X \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I$
    * $I \otimes X \otimes I \otimes X \otimes X \otimes I \otimes X \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I$
    * $I \otimes I \otimes X \otimes I \otimes X \otimes I \otimes I \otimes X \otimes I \otimes I \otimes I \otimes I \otimes I$
    * $I \otimes I \otimes I \otimes I \otimes I \otimes X \otimes I \otimes I \otimes X \otimes I \otimes X \otimes I \otimes I$
    * $I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes X \otimes I \otimes X \otimes X \otimes I \otimes X \otimes I$
    * $I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes X \otimes I \otimes X \otimes I \otimes I \otimes X$

* $Z$ stabilizers
    * $Z \otimes Z \otimes I \otimes Z \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I$
    * $I \otimes Z \otimes Z \otimes I \otimes Z \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I$
    * $I \otimes I \otimes I \otimes Z \otimes I \otimes Z \otimes Z \otimes I \otimes Z \otimes I \otimes I \otimes I \otimes I$
    * $I \otimes I \otimes I \otimes I \otimes Z \otimes I \otimes Z \otimes Z \otimes I \otimes Z \otimes I \otimes I \otimes I$
    * $I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes Z \otimes I \otimes Z \otimes Z \otimes I$
    * $I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes Z \otimes I \otimes Z \otimes Z$

Adding these to our `QECStab` object as strings:


In [2]:
surface_code.add_stab("XIIXIXIIIIIII")
surface_code.add_stab("IXIXXIXIIIIII")
surface_code.add_stab("IIXIXIIXIIIII")
surface_code.add_stab("IIIIIXIIXIXII")
surface_code.add_stab("IIIIIIXIXXIXI")
surface_code.add_stab("IIIIIIIXIXIIX")

surface_code.add_stab("ZZIZIIIIIIIII")
surface_code.add_stab("IZZIZIIIIIIII")
surface_code.add_stab("IIIZIZZIZIIII")
surface_code.add_stab("IIIIZIZZIZIII")
surface_code.add_stab("IIIIIIIIZIZZI")
surface_code.add_stab("IIIIIIIIIZIZZ")

Since the code has $k=1$ logical qubit, we must have one logical $\bar{X}$ and logical $\bar{Z}$ operator for this logical qubit. For the surface code, they are defined as follows:

<img src="./Figs/LER-surfacelogicals.svg" width="600">

As a tensor product, these are:
* $\bar{X} = X \otimes X \otimes X \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I \otimes I$
* $\bar{Z} = Z \otimes I \otimes I \otimes I \otimes I \otimes Z \otimes I \otimes I \otimes I \otimes I \otimes Z \otimes I \otimes I$

We add these to our `StabCode` object, where the first argument denotes the index of the logical qubit (we only have one logical qubit at index 0 here) and the second denotes the logical operator as a string:

In [3]:
#surface_code.set_logical_X(0, "XXXIIIIIIIIII")
surface_code.set_logical_Z(0, "ZIIIIZIIIIZII")

Error propagation does not reduce the effective code distance for the unrotated surface code. Therefore, we can do standard syndrome extraction. As an example, this looks like the following for an $X$ stabilizer.

<img src="./Figs/LER-surfacesyndrome.svg" width="300">

In other words, that $X$ stabilizer takes the $X$ parity of physical qubits 1, 3, 4 and 6. Then, the syndrome qubit is measure in $Z$ (computational / $\{\ket{0}, \ket{1}\}$) basis. This syndrome will detect $Z$ errors.

An analogous measurement scheme is done for $X$ stabilizers. This is all encompassed in the "standard" parity measurement scheme (other options: "Shor", "Knill", "Flag"):


In [4]:
surface_code.scheme = "Standard"

Additionally, we perform two rounds of syndrome extraction. This means that each stabilizer is measured twice. At the end, we will take the parity of those two syndrome outcomes. We call this result a detector outcome. If this result is even, then the two rounds agreed; otherwise they disagreed. 

In [5]:
surface_code.rounds = 2

Given the stabilizers and measurement scheme that we specified, we can now compile the stabilizer code to a quantum circuit (in Stim) corresponding with a single function call to `construct_circuit()`. This circuit will have 13 physical qubits, 12 syndrome qubits (one for each $X$ and $Z$ stabilizer -- between each round of syndrome extraction, we measure the syndrome qubit and reset it's value for the next round) and syndrome extraction will be done given the stabilizers we specified. 

In [6]:
surface_code.construct_circuit()

A clean way to make sure that out circuit was compiled correctly is to see the intermediate representation (IR). We can do so with a function `show_IR()`. The key is below, where `X`, `Y`, `Z` are some placeholders for numbers:
* `cX = Prop[r, s] str`: The `X`th stabilizer check during syndrome extraction round `r` corresponding to stabilizer `s`, with string representation `str`.
* `dX = Parity cY cZ`: The `X`th detector is the parity of checks `Y` and `Z`
* `oX = Parity cY`: The `X`th detector is the exact same result as check `Y`.

In [7]:
surface_code.show_IR()

c0 = Prop[r=0, s=0] XIIXIXIIIIIII
c1 = Prop[r=0, s=1] IXIXXIXIIIIII
c2 = Prop[r=0, s=2] IIXIXIIXIIIII
c3 = Prop[r=0, s=3] IIIIIXIIXIXII
c4 = Prop[r=0, s=4] IIIIIIXIXXIXI
c5 = Prop[r=0, s=5] IIIIIIIXIXIIX
c6 = Prop[r=0, s=6] ZZIZIIIIIIIII
c7 = Prop[r=0, s=7] IZZIZIIIIIIII
c8 = Prop[r=0, s=8] IIIZIZZIZIIII
c9 = Prop[r=0, s=9] IIIIZIZZIZIII
c10 = Prop[r=0, s=10] IIIIIIIIZIZZI
c11 = Prop[r=0, s=11] IIIIIIIIIZIZZ
c12 = Prop[r=1, s=0] XIIXIXIIIIIII
d0 = Parity c0 c12
c13 = Prop[r=1, s=1] IXIXXIXIIIIII
d1 = Parity c1 c13
c14 = Prop[r=1, s=2] IIXIXIIXIIIII
d2 = Parity c2 c14
c15 = Prop[r=1, s=3] IIIIIXIIXIXII
d3 = Parity c3 c15
c16 = Prop[r=1, s=4] IIIIIIXIXXIXI
d4 = Parity c4 c16
c17 = Prop[r=1, s=5] IIIIIIIXIXIIX
d5 = Parity c5 c17
c18 = Prop[r=1, s=6] ZZIZIIIIIIIII
d6 = Parity c6 c18
c19 = Prop[r=1, s=7] IZZIZIIIIIIII
d7 = Parity c7 c19
c20 = Prop[r=1, s=8] IIIZIZZIZIIII
d8 = Parity c8 c20
c21 = Prop[r=1, s=9] IIIIZIZZIZIII
d9 = Parity c9 c21
c22 = Prop[r=1, s=10] IIIIIIIIZIZZI
d10 = Parity

Additionally, one can export this stabilizer circuit to STIM format directly:

In [8]:
print(surface_code.stimcirc)

R 13
H 0
CX 0 13
H 0 3
CX 3 13
H 3 5
CX 5 13
H 5
M 13
R 14
H 1
CX 1 14
H 1 3
CX 3 14
H 3 4
CX 4 14
H 4 6
CX 6 14
H 6
M 14
R 15
H 2
CX 2 15
H 2 4
CX 4 15
H 4 7
CX 7 15
H 7
M 15
R 16
H 5
CX 5 16
H 5 8
CX 8 16
H 8 10
CX 10 16
H 10
M 16
R 17
H 6
CX 6 17
H 6 8
CX 8 17
H 8 9
CX 9 17
H 9 11
CX 11 17
H 11
M 17
R 18
H 7
CX 7 18
H 7 9
CX 9 18
H 9 12
CX 12 18
H 12
M 18
R 19
CX 0 19 1 19 3 19
M 19
R 20
CX 1 20 2 20 4 20
M 20
R 21
CX 3 21 5 21 6 21 8 21
M 21
R 22
CX 4 22 6 22 7 22 9 22
M 22
R 23
CX 8 23 10 23 11 23
M 23
R 24
CX 9 24 11 24 12 24
M 24
R 13
H 0
CX 0 13
H 0 3
CX 3 13
H 3 5
CX 5 13
H 5
M 13
R 14
H 1
CX 1 14
H 1 3
CX 3 14
H 3 4
CX 4 14
H 4 6
CX 6 14
H 6
M 14
R 15
H 2
CX 2 15
H 2 4
CX 4 15
H 4 7
CX 7 15
H 7
M 15
R 16
H 5
CX 5 16
H 5 8
CX 8 16
H 8 10
CX 10 16
H 10
M 16
R 17
H 6
CX 6 17
H 6 8
CX 8 17
H 8 9
CX 9 17
H 9 11
CX 11 17
H 11
M 17
R 18
H 7
CX 7 18
H 7 9
CX 9 18
H 9 12
CX 12 18
H 12
M 18
R 19
CX 0 19 1 19 3 19
M 19
R 20
CX 1 20 2 20 4 20
M 20
R 21
CX 3 21 5 21 6 21 8 21
M 21
R 22
CX

## **Part 2: Testing Logical Error Rates**

* Our encoded qubit in the surface code will eventually be run in a noisy environment. This means that random unitary errors can be injected with probability $p$ in any qubit, in any location, at any point in the circuit. 
* Every so often, we will perform our 2 rounds of syndrome extraction (SE) for each stabilizer. This will output 12 detector outcomes characterizing the type of error that occured. 
* We pass these outcomes into a *decoder* which will determine a candidate correction to fix that error.

$$\text{1 logical qubit} \xrightarrow{\rm encode} \llbracket 13, 1, 3 \rrbracket\text{ surface code} \xrightarrow{\rm noise} \text{errored code} \xrightarrow{\text{2 SE rounds}} \text{12 detector outcomes} \xrightarrow{\rm decoder} \text{``corrected'' code}$$

What we are interested in is how often this decoder fails (i.e., what if the "corrected" code is not the same as the original code?). Let's put this in math:
* Let $\ket{\psi}$ be the state of the logical qubit.
* Let $\ket{\bar{\psi}}$ be the encoded state of the logical qubit as a $\llbracket 13, 1, 3\rrbracket$ surface code.
* Let $E$ be the unitary error that happens.
* Let $C$ be the correction proposed by the decoder

We then have:

$$\ket{\psi} \xrightarrow{\rm encode} \ket{\bar{\psi}} \xrightarrow{\rm noise} E\ket{\bar{\psi}} \xrightarrow{\text{2 SE rounds + decoder}} CE\ket{\bar{\psi}}$$

A logical error *did not* occur if $CE \in \mathcal{S}$, which means that the correction applied on the error is generated by the stabilizer group, meaning that it acts trivially on the codespace. A logical error *did* occur if $CE \in \bar{\mathcal{P}}$, which means that the correction applied on the error creates a logical Pauli operator.

Suppose we encode our logical qubit, perform 2 rounds of syndrome extraction, and decode $N$ times. Let's say that out of these $N$ runs, a logical error occurs $K$ times. The logical error rate as a function of physical error rate $p$ is defined as:

$$\begin{equation} P_L(p) = \lim_{N \rightarrow \infty}\dfrac{K}{N}. \end{equation}$$

Equivalently, this is simply the probability that the decoder produces a logical fault.


Before we estimate logical error rates, let us create our noise model. We set the rate of a physical error to be $p = 1/1000$. We then create a `NoiseModel` object which simply encompasses the fact that we insert random Pauli errors, in random locations, on random qubits, with probability $p$.

In [9]:
from scalerqec.QEC.noisemodel import NoiseModel

p = 0.001
noise_model = NoiseModel(p)

## Methods for estimating Logical Error Rate

1. **Monte-Carlo Sampling**
    * This method is simple. Take Equation $(1)$, choose $K$ to be high (say 100) and then brute force the formula $K/N$.
    * Basically, we keep sampling until 100 logical errors occur.
    * We can simulate that for the surface code as follows:
    
        - Create a calculator object specifying $K=100$.
        - Use `calculate_LER_from_StabCode()` to estimate the logical error rate of our surface code using our specified noise model. Also, we repeat 3 times and take the average for a closer estimate.

In [10]:
from scalerqec.Monte.monteLER import MonteLERcalc

calc = MonteLERcalc(MIN_NUM_LE_EVENT=100)
calc.calculate_LER_from_StabCode(surface_code, noise_model, repeat=3)

Time(STIM):  2.54(+0.39)*10^-2
PL(STIM):  2.94(+0.14)*10^-2
Nerror(STIM):  1.39(+0.21)*10^2
Sample(STIM):  4.73(+0.59)*10^3


2. **Symbolic Solution**
    - This method will use a dynamic programming formula (exponential complexity) to calculate the exact logical error rate.
    - Similarly, we create a calculator object, input our `StabCode` circuit and noise model, and the LER is computed directly.

In [ ]:
from scalerqec.Symbolic.symbolicLER import SymbolicLERcalc

calc = SymbolicLERcalc()        
ler = calc.calculate_LER_from_StabCode(surface_code, noise_model)
print("Symbolic LER: ", ler)

---Step2: Generate the prediction table---
Total detector outcome:  4096
---Step2: construction QEPG--------------
---Step3: calculating error indices--------------
---Step4: dynamic algorithm--------------


3. **Stratified Curve Fitting**
    - d

